# Multiword expressions identification and extraction

The task shows two simple methods useful for identifying multiword expressions (MWE) in corpora.

## Tasks

1. Use SpaCy [tokenizer API](https://spacy.io/api/tokenizer) to tokenize the text from the law corpus.
https://spacy.io/usage

python -m venv .env

source .env/bin/activate

conda install -c conda-forge spacy

python -m spacy download pl_core_news_sm

In [1]:
import os
import spacy
from collections import Counter
nlp = spacy.load("pl_core_news_sm")


def tokenize_data():
    docs = []
    directory = '../ustawy/'
    file_list = os.listdir(os.getcwd() + '/' + directory)

    for filename in file_list:
        with open(os.path.join(directory + filename), 'r') as file:
            infile = file.read()
            doc_ = nlp(infile, disable=["tagger", "parser"])
            docs.append([filename, doc_])
    return docs

tokenized = tokenize_data()

tokens = []
for doc in tokenized:
    for token in doc[1]:
        # print(token.text, token.pos_)
        tokens.append(token.text.lower())

2. Compute **bigram** counts of downcased tokens.  Given the sentence: "The quick brown fox jumps over the
   lazy dog.", the bigram counts are as follows:
   1. "the quick": 1
   1. "quick brown": 1
   1. "brown fox": 1
   1. ...
   1. "dog .": 1

In [2]:
def compute_bigrams(tokens):
    bigrams = {}
    for i in range(0, len(tokens) -1):
        t1 = tokens[i]
        t2 = tokens[i + 1]
        if (t1, t2) not in bigrams:
            bigrams[(t1, t2)] = 1
        else:
            bigrams[(t1, t2)] += 1
    return bigrams

bigrams = compute_bigrams(tokens)
bigrams_sorted = dict(sorted(bigrams.items(), key=lambda b : b[1], reverse=True))
list(bigrams_sorted.items())[:10]

[(('art', '.'), 83778),
 (('ust', '.'), 53552),
 (('.', '\n'), 49741),
 (('poz', '.'), 45198),
 ((',', 'poz'), 39655),
 (('-', '-'), 36542),
 (('r', '.'), 33015),
 (('w', 'art'), 30170),
 (('.', '1'), 29734),
 ((',', 'o'), 28739)]

3. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries **after**
   computing the bigram counts.

In [3]:
def discard_other_than_leters(bigrams):
    bigrams_filtered = dict(filter(lambda a: a[0][0].isalpha() and a[0][1].isalpha(), bigrams.items()))
    return bigrams_filtered

bigrams_filtered = discard_other_than_leters(bigrams_sorted)
list(bigrams_filtered.items())[:10]


[(('w', 'art'), 30170),
 (('mowa', 'w'), 27649),
 (('w', 'ust'), 22238),
 (('których', 'mowa'), 12973),
 (('o', 'których'), 12604),
 (('otrzymuje', 'brzmienie'), 9168),
 (('z', 'dnia'), 8989),
 (('którym', 'mowa'), 8689),
 (('o', 'którym'), 8525),
 (('do', 'spraw'), 8215)]

4. Use [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) to compute the measure
   for all pairs of words.

In [4]:
import math
def compute_freq_list(tokenized):
    from collections import defaultdict
    freqlist = defaultdict(int)
    for doc in tokenized:
        for token in doc[1]:
            freqlist[token.text.lower()]+=1
    return freqlist

freq_list = compute_freq_list(tokenized)

def calc_bigrams_pmi(bigrams, freq_list):
    words_count = sum(list(map(lambda item: item[1], freq_list.items())))
    bigrams_pmi = {}
    for item in bigrams.items():
        bigram = item[0]
        count = item[1]
        bigrams_pmi[bigram] = math.log(count * words_count / (freq_list[bigram[0]] * freq_list[bigram[1]]))
    return bigrams_pmi

bigrams_pmi = calc_bigrams_pmi(bigrams_filtered, freq_list)
list(bigrams_pmi.items())[:5]

[(('w', 'art'), 2.345249554756128),
 (('mowa', 'w'), 3.3266868664983607),
 (('w', 'ust'), 2.4864642934945858),
 (('których', 'mowa'), 4.987799469251011),
 (('o', 'których'), 4.147777751557646)]

5. Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [5]:
bigrams_pmi_sorted = dict(sorted(bigrams_pmi.items(), key=lambda item: item[1], reverse=True))
top10_pmi_bigrams_sorted = list(bigrams_pmi_sorted)[:10]
top10_pmi_bigrams_sorted

[('sezony', 'wegetacyjne'),
 ('pasiekę', 'umiejscawia'),
 ('varroa', 'jacobsoni'),
 ('agrotechniki', 'nienaruszającej'),
 ('kologisk', 'jordburg'),
 ('ef', 'kontrolordning'),
 ('ökologische', 'agrarwirtschaft'),
 ('biologi', 'gewria'),
 ('suthma', 'eegcou'),
 ('eegcou', 'eok')]

6. Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5
   occurrences).

In [6]:
bigrams_pmi_sorted_5filtered = dict(filter(lambda a: bigrams_filtered[a[0]] >= 5, bigrams_pmi_sorted.items()))
top10_pmi_bigrams_sorted_5filtered = list(bigrams_pmi_sorted_5filtered)[:10]
top10_pmi_bigrams_sorted_5filtered

[('ręcznego', 'miotacza'),
 ('grzegorz', 'schetyna'),
 ('świeckie', 'przygotowujące'),
 ('młynki', 'młotkowe'),
 ('młyny', 'kulowe'),
 ('otworami', 'wiertniczymi'),
 ('klęskami', 'żywiołowymi'),
 ('środa', 'wlkp'),
 ('stajnią', 'wyścigową'),
 ('zaszkodzić', 'wynikom')]

7. Use [log likelihood ratio](http://tdunning.blogspot.com/2008/03/surprise-and-coincidence.html) (LLR) to compute the measure
   for all pairs of words.

> Skorzystałem z: https://github.com/tdunning/python-llr/blob/master/llr.py

In [7]:
def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k/total + (k==0)) for k in counts])

def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11+k12, k21+k22]) +
                denormEntropy([k11+k21, k12+k22]) -
                denormEntropy([k11, k12, k21, k22]))

def calc_bigrams_llr(bigrams, freq_list):
    words_count = sum(list(map(lambda item: item[1], freq_list.items())))
    bigrams_llr = {}
    for item in bigrams.items():
        bigram = item[0]
        count = item[1]
        b0 = bigram[0]
        b1 = bigram[1]
        b0_and_b1 = count
        b0_wo_b1 = freq_list[b0] - count
        b1_wo_b0 = freq_list[b1] - count
        n_b0_n_b1 = words_count - freq_list[b0] - freq_list[b1]
        bigrams_llr[bigram] = llr_2x2(b0_and_b1, b1_wo_b0, b0_wo_b1, n_b0_n_b1)

    return bigrams_llr

bigrams_llr = calc_bigrams_llr(bigrams_filtered, freq_list)
list(bigrams_llr.items())[:5]


[(('w', 'art'), 101178.90501589281),
 (('mowa', 'w'), 180285.02858320414),
 (('w', 'ust'), 81238.3993172762),
 (('których', 'mowa'), 123606.48304089962),
 (('o', 'których'), 94268.34465772077)]

8. Sort the word pairs according to that measure in the descending order and display top 10 entries.

In [8]:
bigrams_llr_sorted = dict(sorted(bigrams_llr.items(), key=lambda item: item[1], reverse=True))
top10_llr_bigrams_sorted = list(bigrams_llr_sorted)[:10]
top10_llr_bigrams_sorted

[('mowa', 'w'),
 ('których', 'mowa'),
 ('otrzymuje', 'brzmienie'),
 ('w', 'art'),
 ('o', 'których'),
 ('którym', 'mowa'),
 ('w', 'ust'),
 ('dodaje', 'się'),
 ('do', 'spraw'),
 ('o', 'którym')]

9. Compute **trigram** counts for the whole corpus and perform the same filtering.

In [9]:
def compute_trigrams(tokens):
    trigrams = {}
    for i in range(0, len(tokens) -2):
        t1 = tokens[i]
        t2 = tokens[i + 1]
        t3 = tokens[i + 2]
        if (t1, t2, t3) not in trigrams:
            trigrams[(t1, t2, t3)] = 1
        else:
            trigrams[(t1, t2, t3)] += 1
    return trigrams

trigrams = compute_trigrams(tokens)
trigrams_sorted = dict(sorted(trigrams.items(), key=lambda b : b[1], reverse=True))
list(trigrams_sorted.items())[:10]


[((',', 'poz', '.'), 39633),
 (('-', '-', '-'), 34641),
 (('w', 'art', '.'), 30162),
 (('ust', '.', '1'), 22618),
 (('w', 'ust', '.'), 22204),
 (('r', '.', 'nr'), 16956),
 (('_', '_', '_'), 16111),
 (('których', 'mowa', 'w'), 12506),
 (('mowa', 'w', 'ust'), 12388),
 ((',', 'o', 'których'), 12009)]

In [10]:
def discard_other_than_leters_in_tri(trigrams):
    trigrams_filtered = dict(filter(lambda a: a[0][0].isalpha() and a[0][1].isalpha() and a[0][2].isalpha(), trigrams.items()))
    return trigrams_filtered

trigrams_filtered = discard_other_than_leters_in_tri(trigrams_sorted)
list(trigrams_filtered.items())[:10]


[(('których', 'mowa', 'w'), 12506),
 (('mowa', 'w', 'ust'), 12388),
 (('o', 'których', 'mowa'), 11714),
 (('mowa', 'w', 'art'), 10995),
 (('którym', 'mowa', 'w'), 8429),
 (('o', 'którym', 'mowa'), 8040),
 (('której', 'mowa', 'w'), 5020),
 (('o', 'której', 'mowa'), 4731),
 (('właściwy', 'do', 'spraw'), 4434),
 (('minister', 'właściwy', 'do'), 4106)]

10. Use PMI (with 5 occurrence threshold) and LLR to compute top 10 results for the trigrams. Devise a method for computing the values, based on the
   results for bigrams.

In [11]:
def calc_trigrams_pmi(trigrams, freq_list):
    words_count = sum(list(map(lambda item: item[1], freq_list.items())))
    trigrams_pmi = {}
    for item in trigrams.items():
        trigram = item[0]
        count = item[1]
        trigrams_pmi[trigram] = math.log(count * words_count / (freq_list[trigram[0]] * freq_list[trigram[1]] * freq_list[trigram[2]]))
    return trigrams_pmi

trigrams_pmi = calc_trigrams_pmi(trigrams_filtered, freq_list)
list(trigrams_pmi.items())[:5]


[(('których', 'mowa', 'w'), -7.261036270580908),
 (('mowa', 'w', 'ust'), -8.36615019573926),
 (('o', 'których', 'mowa'), -6.192991978733537),
 (('mowa', 'w', 'art'), -8.931698088471862),
 (('którym', 'mowa', 'w'), -7.236146725727466)]

In [12]:
trigrams_pmi_sorted = dict(sorted(trigrams_pmi.items(), key=lambda item: item[1], reverse=True))
# top10_pmi_trigrams_sorted = list(trigrams_pmi_sorted)[:10]

In [13]:
trigrams_pmi_sorted_5filtered = dict(filter(lambda a: trigrams_filtered[a[0]] >= 5, trigrams_pmi_sorted.items()))
top10_pmi_trigrams_sorted_5filtered = list(trigrams_pmi_sorted_5filtered)[:10]
top10_pmi_trigrams_sorted_5filtered

[('profilem', 'zaufanym', 'epuap'),
 ('finałowego', 'turnieju', 'mistrzostw'),
 ('przedwczesnego', 'wyrębu', 'drzewostanu'),
 ('potwierdzonym', 'profilem', 'zaufanym'),
 ('piłce', 'nożnej', 'uefa'),
 ('cienką', 'sierścią', 'zwierzęcą'),
 ('szybkiemu', 'postępowi', 'technicznemu'),
 ('turnieju', 'mistrzostw', 'europy'),
 ('grożącą', 'jemu', 'samemu'),
 ('wypalonym', 'paliwem', 'jądrowym')]

> LRR dla trigramów liczę poprzez połączenie bigramu i dodatkowego słowa

In [14]:
def calc_trigrams_llr(trigrams, freq_list):
    words_count = sum(list(map(lambda item: item[1], freq_list.items())))
    trigrams_llr = {}
    for item in trigrams.items():
        trigram = item[0]
        count = item[1]
        b0 = (trigram[0], trigram[1])
        b1 = trigram[2]
        b0_and_b1 = count
        b0_wo_b1 = bigrams[b0] - count
        b1_wo_b0 = freq_list[b1] - count
        n_b0_n_b1 = words_count - freq_list[b0] - freq_list[b1]
        trigrams_llr[trigram] = llr_2x2(b0_and_b1, b1_wo_b0, b0_wo_b1, n_b0_n_b1)

    return trigrams_llr

trigrams_llr = calc_trigrams_llr(trigrams_filtered, freq_list)
list(trigrams_llr.items())[:5]

[(('których', 'mowa', 'w'), 80990.40335285128),
 (('mowa', 'w', 'ust'), 81512.409128526),
 (('o', 'których', 'mowa'), 123580.04397681577),
 (('mowa', 'w', 'art'), 58107.125370708294),
 (('którym', 'mowa', 'w'), 54786.1951494785)]

In [15]:
trigrams_llr_sorted = dict(sorted(trigrams_llr.items(), key=lambda item: item[1], reverse=True))
top10_llr_trigrams_sorted = list(trigrams_llr_sorted)[:10]
top10_llr_trigrams_sorted

[('o', 'których', 'mowa'),
 ('o', 'którym', 'mowa'),
 ('mowa', 'w', 'ust'),
 ('których', 'mowa', 'w'),
 ('mowa', 'w', 'art'),
 ('właściwy', 'do', 'spraw'),
 ('którym', 'mowa', 'w'),
 ('o', 'której', 'mowa'),
 ('w', 'drodze', 'rozporządzenia'),
 ('ustawie', 'z', 'dnia')]

11. Create a table comparing the methods (separate table for bigrams and trigrams).

In [16]:
import pandas as pd
bigrams_results_tab = pd.DataFrame({
    'bigrams pmi' : top10_pmi_bigrams_sorted,
    'bigrams pmi 5 filtered' : top10_pmi_bigrams_sorted_5filtered,
    'bigrams llr' : top10_llr_bigrams_sorted})
bigrams_results_tab

,bigrams pmi,bigrams pmi 5 filtered,bigrams llr
0,"(sezony, wegetacyjne)","(ręcznego, miotacza)","(mowa, w)"
1,"(pasiekę, umiejscawia)","(grzegorz, schetyna)","(których, mowa)"
2,"(varroa, jacobsoni)","(świeckie, przygotowujące)","(otrzymuje, brzmienie)"
3,"(agrotechniki, nienaruszającej)","(młynki, młotkowe)","(w, art)"
4,"(kologisk, jordburg)","(młyny, kulowe)","(o, których)"
5,"(ef, kontrolordning)","(otworami, wiertniczymi)","(którym, mowa)"
6,"(ökologische, agrarwirtschaft)","(klęskami, żywiołowymi)","(w, ust)"
7,"(biologi, gewria)","(środa, wlkp)","(dodaje, się)"
8,"(suthma, eegcou)","(stajnią, wyścigową)","(do, spraw)"
9,"(eegcou, eok)","(zaszkodzić, wynikom)","(o, którym)"


In [18]:
trigrams_results_tab = pd.DataFrame({
    'trigrams pmi filtered' : top10_pmi_trigrams_sorted_5filtered,
    'trigrams lrr' : top10_llr_trigrams_sorted
})
trigrams_results_tab

,trigrams pmi filtered,trigrams lrr
0,"(profilem, zaufanym, epuap)","(o, których, mowa)"
1,"(finałowego, turnieju, mistrzostw)","(o, którym, mowa)"
2,"(przedwczesnego, wyrębu, drzewostanu)","(mowa, w, ust)"
3,"(potwierdzonym, profilem, zaufanym)","(których, mowa, w)"
4,"(piłce, nożnej, uefa)","(mowa, w, art)"
5,"(cienką, sierścią, zwierzęcą)","(właściwy, do, spraw)"
6,"(szybkiemu, postępowi, technicznemu)","(którym, mowa, w)"
7,"(turnieju, mistrzostw, europy)","(o, której, mowa)"
8,"(grożącą, jemu, samemu)","(w, drodze, rozporządzenia)"
9,"(wypalonym, paliwem, jądrowym)","(ustawie, z, dnia)"


12. Answer the following questions:
   1. Why do we have to filter the bigrams, rather than the token sequence?
   > Taki sposób filtrowanie pozwala uniknąć tworzenia bigramów składających się z końca zdania poprzedniego i początku zdania następnego.
   > Również osobne części zdania złożonego rozdzielone przecinkiem nie będą brane pod uwagę.
   2. Which measure (PMI, PMI with filtering, LLR) works better for the bigrams and which for the trigrams?
   > W mojej ocenie (jeśli szukamy uniwersalnych wyrażeń wielowyrazowych) dla bigramów najlepiej sprawdza się metoda pmi filtered.
   > W przypadku trigramów różnica nie jest juz tak wyraźna ale wciąż lepiej sprawuje się metoda pmi filtered.
   > LLR dał wyniki wyraźnie powiązane z charakterem tekstów ustawowych, co może być pożądane w niektórych okolicznościach.
   3. What types of expressions are discovered by the methods.
   > PMI dobrze wykrywa wyrażenia wielowyrazowe takie jak nazwy własne.
   > LLR daje rezultaty charakterystyczne  dla tekstów prawnych.
   4. Can you devise a different type of filtering that would yield better results?
   > Filtrowanie wg ilości wystąpień danego bigramu (lub trigramu) powinno być odpowiednio dobrane do wielkości korpusu.
   > Postanowiłem wypróbować kilka różnych wielkości filtra dla bigramów.

In [22]:
bigrams_pmi_sorted_25filtered = dict(filter(lambda a: bigrams_filtered[a[0]] >= 25, bigrams_pmi_sorted.items()))
top10_pmi_bigrams_sorted_25filtered = list(bigrams_pmi_sorted_25filtered)[:10]

bigrams_pmi_sorted_50filtered = dict(filter(lambda a: bigrams_filtered[a[0]] >= 50, bigrams_pmi_sorted.items()))
top10_pmi_bigrams_sorted_50filtered = list(bigrams_pmi_sorted_50filtered)[:10]

bigrams_pmi_sorted_100filtered = dict(filter(lambda a: bigrams_filtered[a[0]] >= 100, bigrams_pmi_sorted.items()))
top10_pmi_bigrams_sorted_100filtered = list(bigrams_pmi_sorted_100filtered)[:10]

bigrams_pmi_sorted_250filtered = dict(filter(lambda a: bigrams_filtered[a[0]] >= 250, bigrams_pmi_sorted.items()))
top10_pmi_bigrams_sorted_250filtered = list(bigrams_pmi_sorted_250filtered)[:10]

bigrams_pmi_sorted_500filtered = dict(filter(lambda a: bigrams_filtered[a[0]] >= 500, bigrams_pmi_sorted.items()))
top10_pmi_bigrams_sorted_500filtered = list(bigrams_pmi_sorted_500filtered)[:10]

bigrams_pmi_sorted_5000filtered = dict(filter(lambda a: bigrams_filtered[a[0]] >= 5000, bigrams_pmi_sorted.items()))
top10_pmi_bigrams_sorted_5000filtered = list(bigrams_pmi_sorted_5000filtered)[:10]

In [23]:
bigrams_results_tab = pd.DataFrame({
    'big pmi 25' : top10_pmi_bigrams_sorted_25filtered,
    'big pmi 50' : top10_pmi_bigrams_sorted_50filtered,
    'big pmi 100' : top10_pmi_bigrams_sorted_100filtered,
    'big pmi 250' : top10_pmi_bigrams_sorted_250filtered,
    'big pmi 500' : top10_pmi_bigrams_sorted_500filtered,
    'big pmi 5000' : top10_pmi_bigrams_sorted_5000filtered,
})

bigrams_results_tab

,big pmi 25,big pmi 50,big pmi 100,big pmi 250,big pmi 500,big pmi 5000
0,"(socjalistycznych, republik)","(klęsk, żywiołowych)","(materiał, siewny)","(sił, zbrojnych)","(papierów, wartościowych)","(otrzymuje, brzmienie)"
1,"(in, vitro)","(wyścigów, konnych)","(rzeczników, patentowych)","(nadanym, niniejszą)","(niniejszą, ustawą)","(którym, mowa)"
2,"(bronisław, komorowski)","(rygor, natychmiastowej)","(przetworów, mlecznych)","(zdanie, drugie)","(produktu, leczniczego)","(których, mowa)"
3,"(ropy, naftowej)","(cienkiej, sierści)","(walnym, zgromadzeniu)","(dzienniku, urzędowym)","(unii, europejskiej)","(której, mowa)"
4,"(klęski, żywiołowej)","(marek, kuchciński)","(papierami, wartościowymi)","(zagospodarowania, przestrzennego)","(zasięgnięciu, opinii)","(dodaje, się)"
5,"(ewa, kopacz)","(buraków, cukrowych)","(papiery, wartościowe)","(tekstu, jednolitego)","(obrony, narodowej)","(stosuje, się)"
6,"(ogrodu, botanicznego)","(zagospodarowaniu, przestrzennym)","(masie, jednostkowej)","(papierów, wartościowych)","(opieki, zdrowotnej)","(na, podstawie)"
7,"(porze, nocnej)","(sierści, zwierzęcej)","(rzeczpospolita, polska)","(energii, elektrycznej)","(straży, granicznej)","(do, spraw)"
8,"(narodów, zjednoczonych)","(trybunału, konstytucyjnego)","(ordynacja, podatkowa)","(ue, l)","(pozbawienia, wolności)","(o, którym)"
9,"(wysokim, bezrobociem)","(narodowi, polskiemu)","(walne, zgromadzenie)","(osobowości, prawnej)","(ubezpieczeń, społecznych)","(o, których)"


    > Zebrane w tabeli wyniki pokazują, że różne wartości filtru dają dobre wyniki.
    > Bardzo wysoki poziom filtra przybliża wyniki do tych z LLR.

## Hints

1. An n-gram is a sequence containing n words. A unigram is a sequence containing one word,
   a bigram is a sequence containing two consecutive words, etc.
1. *Pointwise mutual information* is used to identify correlated events. It's based on the assumption that the events
   follow normal distribution and that there is a minimal number of occurrences of the words. These assumptions hold
   only for a subset of words.
1. Log likelihood ratio test doesn't have these assumption. This makes it better suited for the task.
1. There is [LLR implementation](https://github.com/tdunning/python-llr) in Python, implemented by Ted Dunning - the
   author of the important work [Accurate Methods for the Statistics of Surprise and
   Coincidence](https://aclweb.org/anthology/J93-1003) which introduces LLR to NLP.
1. The methods presented in this exercise can be also used for the identification of words belonging to a given domain
   (e.g. law, biology, medicine).
1. [SRI LM](http://www.speech.sri.com/projects/srilm/) is useful for computing the counts of n-grams.
   [Gensim](https://radimrehurek.com/gensim/models/phrases.html) also allows
   to compute these values.
1. ElasticSearch has a [shingle token filter](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-shingle-tokenfilter.html)
   which can be used to build the n-grams as well.
1. BTW "multiword expressions" is a mutliword expression itself ;-)